In [1]:
### Loading Libraries
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(sf)
library(sp)
library(geosphere)
library(USAboundaries)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.5.2, GDAL 2.4.2, PROJ 4.8.0



In [2]:
### Reading in Data
fp <- '/nfs/sloanlab004/projects/covid_mobility_proj/data/'
stations <- read.fwf(str_c(fp, "weather/ghcnd-stations.txt"), widths = c(11,9,10,7,3,31,4,4,6), comment.char="")
colnames(stations) <- c("station_id", "lat", "lng", "elev", "state", "name", "GSN", "HCN/CRN", "WMO")
stations %>%
  mutate(station_id = as.character(station_id),
         state = str_sub(as.character(state), 2, 3),
         name = as.character(name)) -> stations

weather <- read_csv(str_c(fp, "weather/2020.csv"), col_names = FALSE)
colnames(weather) <- c("station_id", "date", "type", "value", "X1", "X2", "SFLAG1", "Time")

Parsed with column specification:
cols(
  X1 = col_character(),
  X2 = col_double(),
  X3 = col_character(),
  X4 = col_double(),
  X5 = col_character(),
  X6 = col_character(),
  X7 = col_character(),
  X8 = col_character()
)



In [9]:
stations %>% filter(str_detect(station_id, 'US'))

station_id,long,lat,state
<chr>,<dbl>,<dbl>,<chr>
US10adam002,-98.5493,40.5093,NE
US10adam004,-98.4026,40.4798,NE
US10adam008,-98.2973,40.4953,NE
US10adam010,-98.6297,40.5532,NE
US10adam017,-98.4789,40.6567,NE
US10adam019,-98.5543,40.6114,NE
US10adam022,-98.4299,40.5936,NE
US10adam032,-98.4208,40.3571,NE
US10adam036,-98.4020,40.5609,NE


In [5]:
### Data Processing
# This first part gets the list of "high quality weather stations" with the most complete data
sflags <- weather %>% 
  select(station_id, date, type, value, SFLAG1) %>%
  filter(grepl('US', station_id, fixed = T)) %>%
  filter(type == "PRCP" | type == "TMAX") %>%
  spread(key = type, value = value) %>% 
  select(station_id, date, SFLAG1) %>%
  group_by(station_id, SFLAG1) %>%
  summarize(n = n()) %>%
  group_by(station_id) %>%
  arrange(desc(n)) %>%
  filter(row_number() == 1)

# Preprocessing weather data
weather %>% 
  select(station_id, date, type, value) %>%
  filter(str_detect(station_id, 'US')) %>%
  filter(type == "PRCP" | type == "TMAX") %>%
  spread(key = type, value = value)  %>%
  inner_join(stations %>% select(station_id, lat, lng)) %>%
  inner_join(sflags %>% select(-n)) -> weather

# Preprocessing station list
stations %>%
  semi_join(weather)  %>%
  select(station_id, long = lng, lat, state) -> stations

# Creating sf  version of stations coords
stations_sf <- st_as_sf(stations, coords = c("long", "lat"), crs = 4326) 

# Grabbing county data from USAboundaries library
counties <- us_counties(resolution = 'high')

# Generating countyInfo dataframe, columns are:
# county fips, county name, state
as.data.frame(counties) %>%
  mutate(county_fips = str_c(statefp, countyfp)) %>%
  select(county_fips, county_name = name, state_abbr) %>%
  distinct() -> countyInfo 

# Getting the list of stations inside each county
st_intersects(counties, stations_sf) -> stationsIn
# Adding county fips as keys
names(stationsIn) <- countyInfo$county_fips

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = c("station_id", "lat", "lng")

although coordinates are longitude/latitude, st_intersects assumes that they are planar



In [6]:
# Helper Function to get all stations within a county
mapper <- function(row) {
  key <-  row$county_fips
  return(stations[stationsIn[[key]], ])
}

# Generating dataframe that maps stations to counties
countyInfo %>% 
    select(county_fips) %>% 
    group_by(county_fips) %>%
    do(mapper(.)) -> countyStations

In [7]:
# Freeing up memory
rm(stationsIn, stations_sf, sflags)

In [8]:
# Getting county centroids
st_centroid(counties[1:2]) -> temp

countyInfo %>%
  ungroup() %>%
  mutate(geometry = str_replace(st_as_text(temp$geometry), 'POINT \\(', ''),
         geometry = str_sub(geometry, 1, -2)) %>%
  separate(geometry, sep = ' ', into = c('long', 'lat')) %>%
  mutate(long = as.numeric(long),
         lat  = as.numeric(lat)) -> countyInfo

# Determining which counties don't have weather stations
countyInfo %>%
  anti_join(countyStations, by = 'county_fips') -> missing 

# Determining which counties have < 3 weather stations and adding those ot missing
countyStations %>%
  group_by(county_fips) %>%
  tally() %>%
  filter(n <= 2) %>%
  select(-n) %>%
  left_join(countyInfo) %>%
  bind_rows(missing) -> missing

# Generating distances from country Centroids to each weather station for each county in missing
geoDist  <- distm(matrix(c(missing$long, missing$lat), ncol=2), 
                  matrix(c(stations$long, stations$lat), ncol=2))/1000
colnames(geoDist) <- stations$station_id

# Transform Distance matrix to "long" dataframe
as.data.frame(geoDist) %>%
  mutate(county_fips = missing$county_fips) %>%
  gather(key = 'station_id', 'dist', -county_fips) -> geoDist

# For each county, get closest 3 weather stations
geoDist %>%
  group_by(county_fips) %>%
  arrange(county_fips, dist) %>%
  filter(row_number() < 3) %>%
  left_join(stations) %>%
  filter(dist < 100) %>%
  select(-dist) -> temp

# Adding these to the list of countyStations, and removing duplicates
countyStations %>%
  bind_rows(temp) %>%
  select(-long, -lat, -state) %>%
  distinct() -> countyStations

# Freeing up memory
rm(temp)

Warning message in st_centroid.sf(counties[1:2]):
“st_centroid assumes attributes are constant over geometries of x”
Warning message in st_centroid.sfc(st_geometry(x), of_largest_polygon = of_largest_polygon):
“st_centroid does not give correct centroids for longitude/latitude data”
Joining, by = "county_fips"



In [33]:
# Helper function to compute average PRCP and TMAX per county, removing missing values
mapper2 <- function(county_df) {
  county_df %>%
    left_join(weather) %>%
    group_by(county_fips, date) %>%
    summarize(PRCP = mean(PRCP, na.rm = T),
              TMAX = mean(TMAX, na.rm = T)) -> out
  return(out)
}

# Computing county level weather
countyStations %>%
  group_by(county_fips) %>%
  do(mapper2(.)) -> out

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

...

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "station_id"

Joining, by = "stati

In [35]:
# Removing the latest date
# 2020.csv updates each day at ~2pm, lots of missing data on the latest date as a result
out %>%
  filter(date <= max(date) - 1) -> out

# Spread/Gather to determine which county-dates are missing TMAX
out %>%
  group_by(county_fips) %>%
  select(-PRCP) %>%
  spread(key = date, value = TMAX) %>%
  gather(key = 'date', value = 'TMAX', -county_fips) -> TMAX

# Spread/Gather to determine which county-dates are missing PRCP
out %>%
  group_by(county_fips) %>%
  select(-TMAX) %>%
  spread(key = date, value = PRCP) %>%
  gather(key = 'date', value = 'PRCP', -county_fips) -> PRCP
  
PRCP %>%
  left_join(TMAX) -> out

TMAX %>%
  filter(is.na(TMAX)) %>%
  select(county_fips, date) %>%
  left_join(countyInfo) %>%
  mutate(date = as.numeric(date)) -> missTMAX

PRCP %>%
  filter(is.na(PRCP)) %>%
  select(county_fips, date) %>%
  left_join(countyInfo) %>%
  mutate(date = as.numeric(date)) -> missPRCP

rm(TMAX, PRCP)

Joining, by = c("county_fips", "date")

Joining, by = "county_fips"

Joining, by = "county_fips"



In [36]:
# Getting set counties with missing TMAX
missTMAX %>%
  select(county_fips, long, lat) %>%
  distinct() -> mtc

# Getting set counties with missing PRCP
missPRCP %>%
  select(county_fips, long, lat) %>%
  distinct() -> mpc

# Combining them and removing duplicates
mtc %>%
  bind_rows(mpc) %>%
  distinct() -> missing

rm(mtc, mpc)

# Computing Distance from centroids to weather stations
geoDist <- distm(matrix(c(missing$long, missing$lat), ncol=2), 
                  matrix(c(stations$long, stations$lat), ncol=2))/1000
colnames(geoDist) <- stations$station_id

# Turning matrix to dataframe, filtering out stations too far away
as.data.frame(geoDist) %>%
  mutate(county_fips = missing$county_fips) %>%
  gather(key = 'station_id', 'dist', -county_fips) %>%
  group_by(county_fips) %>%
  arrange(dist)  %>%
  filter(dist <= 100) -> geoDist

# Getting just TMAX
weather %>%
  select(-PRCP) %>%
  filter(!is.na(TMAX)) -> TMAX

# Helper function to fill in missing TMAX
# Averages the 3 (or less closest
fillTMAX <- function(df) {
  print(df$county_fips[1])
  weather %>%
    semi_join(df) %>%
    select(-PRCP) %>%
    filter(!is.na(TMAX)) -> temp
  
  df %>%
    left_join(geoDist) %>%
    inner_join(TMAX) %>%
    group_by(date) %>%
    arrange(dist) %>%
    filter(row_number() <= 3) %>%
    filter(dist <= 100) %>%
    summarize(TMAX.r = mean(TMAX)) -> out
  
  return(out)
}

fillPRCP <- function(df) {
  print(df$county_fips[1])
  weather %>%
    semi_join(df) %>%
    select(-TMAX) %>%
    filter(!is.na(PRCP)) -> temp
  
  df %>%
    left_join(geoDist) %>%
    inner_join(PRCP) %>%
    group_by(date) %>%
    arrange(dist) %>%
    filter(row_number() <= 3) %>%
    filter(dist <= 100) %>%
    summarize(PRCP.r = mean(PRCP)) -> out
  
  return(out)
}

missTMAX %>%
  select(county_fips, date) %>%
  group_by(county_fips) %>%
  do(fillTMAX(.)) -> rTMAX

weather %>%
  select(-TMAX) %>%
  filter(!is.na(PRCP)) -> PRCP

missPRCP %>%
  select(county_fips, date) %>%
  group_by(county_fips) %>%
  do(fillPRCP(.)) -> rPRCP

[1] "01001"


Joining, by = "date"

Joining, by = "county_fips"

Joining, by = c("date", "station_id")



[1] "01009"


Joining, by = "date"

Joining, by = "county_fips"

Joining, by = c("date", "station_id")



[1] "01011"


...

Joining, by = "date"

Joining, by = "county_fips"

Joining, by = c("date", "station_id")



[1] "55121"


Joining, by = "date"

Joining, by = "county_fips"

Joining, by = c("date", "station_id")



[1] "56041"


Joining, by = "date"

Joining, by = "county_fips"

Joining, by = c("date", "station_id")



In [39]:
out %>%
  mutate(date = as.numeric(date)) %>%
  left_join(rTMAX) %>%
  left_join(rPRCP) %>%
  mutate(TMAX = ifelse(is.na(TMAX), TMAX.r/10, TMAX/10),
         PRCP = ifelse(is.na(PRCP), PRCP.r/10, PRCP/10),
         dt = as.Date(as.character(date), format = '%Y%m%d')) %>%
  select(county_fips, dt, PRCP, TMAX) -> out

out %>%
  filter(is.na(PRCP) | is.na(TMAX)) %>%
  select(county_fips) %>% 
  distinct()

out %>%
  anti_join(out %>%
              filter(is.na(PRCP) | is.na(TMAX)) %>%
              select(county_fips) %>% 
              distinct()) -> out

Joining, by = c("county_fips", "date")

Joining, by = c("county_fips", "date")



county_fips
<chr>
02158
02016


Joining, by = "county_fips"



In [43]:
write_csv(out, str_c(fp, 'PROCESSED_DATA/county_weather.csv'))